## Instal·lant dependències

In [ ]:
!pip3 install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip3 install fastai

In [ ]:
import os
import fastai
from fastai import *
from fastai.text import *

import numpy as np
import pandas as pd

from functools import partial

## Load dataset

In [23]:
import pandas as pd

In [30]:
data = pd.read_csv('data.csv', delimiter='|', error_bad_lines=False)
data

,Type,Article
0,SPORTS,Una terrible calamarsada ha castigat aquest di...
1,SPORTS,La bona resposta de l'afició ha acabat donant ...
2,SPORTS,Frida Eldebrink i Giedre Labuckiene (foto) es ...
3,SPORTS,"Barça i Bayern de Munic, els dos únics campion..."
4,SPORTS,L'equip Racing Point de Fórmula 1 va anunciar ...
...,...,...
59,SALUT,Les comarques gironines han registrat 9.263 ca...
60,SALUT,Les proves clíniques de la vacuna contra la co...
61,SALUT,Un equip d'investigadors del Consell Superior ...
62,SALUT,Un estudi dirigit per la Charité - Universität...


In [27]:
data.shape

(64, 2)

## Preprocessing: Tokenitzar i estandaritzarització

In [32]:
data['text'] = data['Article'].str.replace("[^a-zA-Z]", " ")

In [47]:
!pip3 install nltk


You should consider upgrading via the 'pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'nltk'

In [46]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

ModuleNotFoundError: No module named 'nltk'

In [ ]:
stop_words = stopwords.words('catalan')

## Carregar els weights del Language Model

In [ ]:
learn = language_model_learner(data_LM, pretrainned_model=URLs.WT103, drop_mult=0.7)

## Finetuning del Language Model en la nostra data

Primer, abans d'entrenar el classificador (només fent servir l'encoder) fem *fine-tuning* amb la nostra data. Tot i que el model ha estat entrenat amb un dataset de domini-general molt divers, probablement la nostra data té una distribució diferent.

In [ ]:
# entrenem el learner objecte un epoch
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.lr_find()

In [ ]:
learn.recoder.plot()

In [ ]:
learn.save_encoder('encoder')

## Finetuning de l'encoder

Ara toca entrenar el classificador utilitzant només l'encoder. 

In [ ]:
learn = text_classifier_learner(data_class, drop_mult=0.7)
learn.load_encoder('encoder')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one(1, 1e-2)